<a href="https://colab.research.google.com/github/Yogeshkumar170201/Yogeshkumar170201/blob/master/FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install -q kaggle

In [33]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hasyimabdillah/workoutfitness-video

In [ ]:
!apt install unzip

In [ ]:
!unzip /content/workoutfitness-video.zip -d /content/datset

In [ ]:
!pip install mediapipe

In [35]:
import tensorflow as tf
import numpy as np
import cv2
import os
import mediapipe as mp
from google.colab.patches import cv2_imshow
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping

In [36]:
mp_holistic = mp.solutions.holistic

In [37]:
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 10

In [38]:
def landmarks_extraction(path):
  # Declare a list to store video frames.
  frames_list = []
  # Read the Video File using the VideoCapture object.
  video_reader = cv2.VideoCapture(path)

  # Get the total number of frames in the video.
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  # Calculate the the interval after which frames will be added to the list.
  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

  # Iterate through the Video Frames.
  
  for frame_counter in range(SEQUENCE_LENGTH):

      # Set the current frame position of the video.
      video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

      # Reading the frame from the video. 
      success, frame = video_reader.read() 

      # Check if Video frame is not successfully read then break the loop
      if not success:
          break
      # Append the normalized frame into the frames list
      frames_list.append(frame)

  # Release the VideoCapture object. 
  video_reader.release()
  marks = []
  with mp_holistic.Holistic() as holistic:
    for i in frames_list:
      # Convert frame to RGB
      frame = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)

      # Detect poses in frame
      results = holistic.process(frame)
      pose_landmarks = results.pose_landmarks
      # print(pose_landmarks)
      if pose_landmarks is not None:
        landmarks = [(lmk.x, lmk.y, lmk.z) for lmk in pose_landmarks.landmark]
        marks.append(np.array(landmarks))
  # Return the frames list.
  marks = np.array(marks)
  return marks

In [39]:
# Specify the directory containing the dataset. 
DATASET_DIR = "datset"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = os.listdir('/content/datset')
NO_OF_CATEGORIES = 5
CLASSES_LIST = CLASSES_LIST[:NO_OF_CATEGORIES]

In [40]:
def create_dataset():
  marks = []
  labels = []
  for class_index, class_name in enumerate(CLASSES_LIST):
    print(class_index, class_name)
    # Display the name of the class whose data is being extracted.
    print(f'Extracting Data of Class: {class_name}')

    # Get the list of video files present in the specific class name directory.
    files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

    # Iterate through all the files present in the files list.
    for file_name in files_list:

      # Get the complete video path.
      video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

      # Extract the landmarks of the video file.
      landmarks = landmarks_extraction(video_file_path)
      marks.append(np.array(landmarks))
      labels.append(class_index)
  return marks, labels

In [41]:
marks, labels = create_dataset()

0 hammer curl
Extracting Data of Class: hammer curl
1 push-up
Extracting Data of Class: push-up
2 incline bench press
Extracting Data of Class: incline bench press
3 bench press
Extracting Data of Class: bench press
4 hip thrust
Extracting Data of Class: hip thrust


In [42]:
labels = np.array(labels)
marks = np.array(marks)
x = marks
y = labels

<ipython-input-42-021d6fe70071>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  marks = np.array(marks)


In [43]:
print(x.shape)

(187,)


In [44]:
for i in range(0, len(x)):
  if x[i].size!=0:
    x[i] = x[i].reshape(x[i].shape[0], -1)

In [45]:
print(x.shape)

(187,)


In [46]:
for i in reversed(range(len(x))):
  if x[i].size==0:
    x = np.delete(x, i)
    y = np.delete(y, i)

In [47]:
for i in reversed(range(len(x))):
  if x[i].shape[0]<SEQUENCE_LENGTH:
    x = np.delete(x, i)
    y = np.delete(y, i)
  elif x[i].shape[0]>=SEQUENCE_LENGTH:
    x[i] = x[i][:SEQUENCE_LENGTH, ]

In [48]:
print(x.shape)
print(y.shape)

(132,)
(132,)


In [49]:
z = y
print(y.shape)

(132,)


In [50]:
z = tf.keras.utils.to_categorical(z).astype(int)
print(z.shape)

(132, 5)


In [51]:
print(z)

[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]

In [52]:
# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(SEQUENCE_LENGTH, 99)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NO_OF_CATEGORIES, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               116736    
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_6 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)               

In [53]:
X_train, X_test, y_train, y_test = train_test_split(x, z, test_size=0.25, random_state=42)

In [54]:
for i in range(len(X_train)):
  X_train[i] = tf.constant(X_train[i])
for i in range(len(X_test)):
  X_test[i] = tf.constant(X_test[i])

In [55]:
x_train = tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)
x_test = tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)

for i in range(len(X_train)):
  x_train = x_train.write(i, tf.constant(X_train[i]))

for i in range(len(X_test)):
  x_test = x_test.write(i, tf.constant(X_test[i]))
y_train = tf.constant(y_train)
y_test = tf.constant(y_test)
print(type(x_train))
print(type(x_test))

<class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>
<class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>


In [56]:
x_train = x_train.stack()
x_test = x_test.stack()
y_train = y_train.stack()
y_test = y_test.stack()

AttributeError: ignored

In [57]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=500, batch_size=40, callbacks=[early_stopping])

Epoch 1/500
3/3 [==============================] - 15s 266ms/step - loss: 1.6087 - accuracy: 0.2525 - val_loss: 1.5661 - val_accuracy: 0.2424
Epoch 2/500
3/3 [==============================] - 0s 20ms/step - loss: 1.5583 - accuracy: 0.3030 - val_loss: 1.5621 - val_accuracy: 0.2424
Epoch 3/500
3/3 [==============================] - 0s 18ms/step - loss: 1.5408 - accuracy: 0.2828 - val_loss: 1.5068 - val_accuracy: 0.2424
Epoch 4/500
3/3 [==============================] - 0s 18ms/step - loss: 1.4657 - accuracy: 0.4141 - val_loss: 1.4299 - val_accuracy: 0.5758
Epoch 5/500
3/3 [==============================] - 0s 19ms/step - loss: 1.4405 - accuracy: 0.3838 - val_loss: 1.3912 - val_accuracy: 0.4545
Epoch 6/500
3/3 [==============================] - 0s 19ms/step - loss: 1.3657 - accuracy: 0.4646 - val_loss: 1.3077 - val_accuracy: 0.5455
Epoch 7/500
3/3 [==============================] - 0s 19ms/step - loss: 1.3894 - accuracy: 0.4747 - val_loss: 1.2208 - val_accuracy: 0.5758
Epoch 8/500
3/3 [=

In [58]:
loss, accuracy = model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 16ms/step - loss: 0.4329 - accuracy: 0.9394


In [59]:
print(loss)
print(accuracy)

0.4328698217868805
0.939393937587738


In [60]:
y_pred = model.predict(x_test)

2/2 [==============================] - 1s 6ms/step


In [61]:
print(y_test)

tf.Tensor(
[[0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]], shape=(33, 5), dtype=int64)


In [62]:
print(y_pred.shape)

(33, 5)


In [64]:
y_predd = []
for i in range(0, y_pred.shape[0]):
  label = 0
  val = 2.2250738585072014e-308
  for j in range(0, NO_OF_CATEGORIES):
    if y_pred[i][j] > val:
      val = y_pred[i][j]
      label = j
  y_predd.append(label)

In [65]:
y_true = []
for i in range(0, y_pred.shape[0]):
  for j in range(0, NO_OF_CATEGORIES):
    if y_test[i][j] == 1:
      y_true.append(j)

In [66]:
print(y_predd)

[1, 2, 1, 1, 2, 3, 1, 4, 1, 1, 1, 1, 1, 0, 3, 3, 1, 2, 1, 1, 1, 0, 1, 1, 3, 2, 4, 3, 3, 0, 0, 1, 2]


In [67]:
print(y_true)

[1, 2, 1, 1, 2, 3, 1, 4, 1, 1, 1, 1, 1, 0, 3, 3, 1, 2, 1, 1, 1, 0, 1, 0, 3, 2, 4, 3, 3, 0, 0, 4, 2]


In [68]:
cm = confusion_matrix(y_true, y_predd)

In [69]:
print(classification_report(y_true, y_predd))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.88      1.00      0.93        14
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         6
           4       1.00      0.67      0.80         3

    accuracy                           0.94        33
   macro avg       0.97      0.89      0.92        33
weighted avg       0.95      0.94      0.94        33



In [70]:
!pip install fastdtw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp39-cp39-linux_x86_64.whl size=534346 sha256=56ff265f9af1b93dc7360c61a6108a69927bc6f02d3beaaff0c0c6414d4ccf74
  Stored in directory: /root/.cache/pip/wheels/1f/a1/63/bfd0fddb5bf0b59f564872e29272cee8a2de0cd745d88fede5
Successfully built fastdtw


In [71]:
import cv2
import numpy as np

def extract_features(video_path, num_frames):
    # Load the video using OpenCV
    cap = cv2.VideoCapture(video_path)
    
    # Initialize empty list to store the features
    features = []
    
    # Loop through each frame of the video
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        
        # Break the loop if there are no more frames
        if not ret:
            break
        
        # Resize the frame to 112x112 pixels
        frame = cv2.resize(frame, (112, 112))
        
        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Stack 3 copies of the grayscale frame to create a 3-channel image
        frame_3channel = np.stack((frame_gray,)*3, axis=-1)
        
        # Normalize the pixel values between -1 and 1
        frame_norm = (frame_3channel - 128) / 128
        
        # Append the normalized frame to the list of features
        features.append(frame_norm)
        
        # If we have collected enough frames, break out of the loop
        if len(features) == num_frames:
            break
    
    # Pad or truncate the frames to match the fixed number of frames
    if len(features) < num_frames:
        num_pad_frames = num_frames - len(features)
        padding = np.zeros((num_pad_frames, 112, 112, 3))
        features = np.concatenate((features, padding), axis=0)
    elif len(features) > num_frames:
        features = features[:num_frames]
    
    # Convert the list of features to a numpy array
    features = np.array(features)
    
    # Reshape the array to (1, num_frames, height, width, channels)
    features = np.reshape(features, (1, features.shape[0], 112, 112, 3))
    
    # Return the extracted features
    return features

In [176]:
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

def align_features(f1, f2):
    # Reshape the feature arrays
    f1 = np.reshape(f1, (f1.shape[0], -1))
    f2 = np.reshape(f2, (f2.shape[0], -1))

    # Compute the pairwise cosine distances between the features
    distances = pairwise_distances(f1, f2, metric='cosine')

    # Use the Hungarian algorithm to find the optimal assignment of features
    row_ind, col_ind = linear_sum_assignment(distances)

    # Sort the assignments in ascending order of row indices
    idx = np.argsort(row_ind)
    row_ind, col_ind = row_ind[idx], col_ind[idx]

    # Select the corresponding features from f1 and f2
    aligned_f1, aligned_f2 = f1[row_ind], f2[col_ind]

    return aligned_f1, aligned_f2


In [177]:
from sklearn.metrics.pairwise import cosine_similarity

def sim_cosine(video1_aligned, video2_aligned):
  similarity = cosine_similarity(video1_aligned, video2_aligned)

  return similarity


In [158]:
def find_category(path):
  test_marks = landmarks_extraction(path)
  test_marks = np.array(test_marks)
  test_marks = test_marks.reshape(test_marks.shape[0], -1)
  test_marks = test_marks.reshape(1, SEQUENCE_LENGTH, 99)
  test_marks = tf.constant(test_marks)
  test_category = model.predict(test_marks)
  highest_index = np.argmax(test_category[0])
  return highest_index

In [131]:
path_of_testvideo =  '/content/pushup2.mp4'

In [161]:
cat = find_category(path_of_testvideo)
print(CLASSES_LIST[cat])

1/1 [==============================] - 0s 47ms/step
push-up


In [180]:
def excercise_right(path, cat):
  mx_sim = 0.00
  path_cat = os.path.join('/content/datset/'+CLASSES_LIST[cat]+'/')
  videos_list = os.listdir(path_cat)
  for i in range(0, len(videos_list)):
    video_path = os.path.join(path_cat, videos_list[i])
    f1 = extract_features(video_path, 100)
    f2 = extract_features(path_of_testvideo, 100)
    align_f1, align_f2 = align_features(f1, f2)
    sim = sim_cosine(align_f1, align_f2)
    if sim>mx_sim:
      mx_sim = sim
  return mx_sim

In [182]:
mx_sim = excercise_right(path_of_testvideo, cat)
print(mx_sim)

[[0.91714401]]


In [184]:
if mx_sim > 0.9:
  print('Excercise is right')
else:
  print('Excercise is not right')

Excercise is right
